In [1]:
import sys
import time
from pathlib import Path
import ssl

sys.path.append(str(Path('../').resolve()))
assert 'python-nostr' in [Path(path).name for path in sys.path]

from nostr.key import PrivateKey, PublicKey
from client import Client

## I'm loading my key from disk

In [2]:
key_file = Path('./private_key.txt') # saving a private key for testing
if key_file.exists():
    with open(key_file, 'r') as f:
        pk_hex = f.read()
        private_key = PrivateKey.from_hex(pk_hex)
    public_key = private_key.public_key
    print(f'Loaded private key from {key_file}')
    print(f'Public key: {public_key.bech32()}')
else:
    private_key = PrivateKey()
    public_key = private_key.public_key
    print(f"Generated new private key")
    print(f"Public key: {public_key.bech32()}")
    pk_hex = private_key.hex()
    with open(key_file, 'w') as f:
        f.write(pk_hex)

Loaded private key from private_key.txt
Public key: npub19nqc7plddvevd8sgfpahzuput0q9hwrfhk8crj9k73fpg8qwjx3sk6224n


In [7]:
# load client with no keys
client = Client(ssl_options={'cert_reqs': ssl.CERT_NONE})

# load client with public key only
client = Client(ssl_options={'cert_reqs': ssl.CERT_NONE}, public_key_hex=public_key.hex())
assert client.public_key.hex() == public_key.hex()
assert client.private_key is None

# load client with public key and private key
client = Client(ssl_options={'cert_reqs': ssl.CERT_NONE}, public_key_hex=public_key.hex(), private_key_hex=pk_hex)
assert client.public_key.hex() == public_key.hex()
assert client.private_key.hex() == pk_hex

# load client with private key only
client = Client(ssl_options={'cert_reqs': ssl.CERT_NONE}, private_key_hex=pk_hex)
assert client.public_key.hex() == public_key.hex()
assert client.private_key.hex() == pk_hex


logged in as public key
	bech32: npub1sufw80nl4e5yz3dzyqqw3tzmshdjtpvsgguq0vmuz85pjzyps24seyql8f
	hex: 8712e3be7fae684145a22000e8ac5b85db258590423807b37c11e819088182ab
logged in as public key
	bech32: npub19nqc7plddvevd8sgfpahzuput0q9hwrfhk8crj9k73fpg8qwjx3sk6224n
	hex: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3
logged in as public key
	bech32: npub19nqc7plddvevd8sgfpahzuput0q9hwrfhk8crj9k73fpg8qwjx3sk6224n
	hex: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3
logged in as public key
	bech32: npub19nqc7plddvevd8sgfpahzuput0q9hwrfhk8crj9k73fpg8qwjx3sk6224n
	hex: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3


# client
instantiating a basic client and testing connections

In [8]:
client = Client(ssl_options={'cert_reqs': ssl.CERT_NONE})

with client:
    print(client.relay_manager.connection_statuses)
    print('exiting client')
print(client.relay_manager.connection_statuses)

logged in as public key
	bech32: npub10ycmr932zl70mxv66uzp4ryketmkku4728t38knv0ngp6zp6m2fsqrmu07
	hex: 7931b1962a17fcfd999ad7041a8c96caf76b72be51d713da6c7cd01d083ada93


/Users/ryanarmstrong/Documents/01_Programming/GitHub/python-nostr/example_client/client.py:101: UserWarning: could not connect to wss://nostr.zebedee.cloud... removing relay.
  warnings.warn(


{'wss://nostr-2.zebedee.cloud': True, 'wss://relay.damus.io': True}
exiting client
{'wss://nostr-2.zebedee.cloud': False, 'wss://relay.damus.io': False}


# make a simple text input client

we will instantiate it with the proper options first

In [4]:
pk_hex

'7fd7a07937911dd3c5a18a6a0cd10d896d7a31eb2613d3b0a57b4f5b101cca27'

In [12]:
from client import TextInputClient
import ssl

text_client = TextInputClient(ssl_options={'cert_reqs': ssl.CERT_NONE}, allow_duplicates=False)

could not generate private key from input. generating a new random key
generated new private key: f7884ad2cc3908c9c710deaf16f4d08c9383e1cf234b67f30a6e5b98497bf8c6
logged in as public key
	bech32: npub1eudk00kj3j77v8acj0lse9egl6k5ylmf9jh2y60rnmu2j74ppqmsw9dz7m
	hex: cf1b67bed28cbde61fb893ff0c9728fead427f692caea269e39ef8a97aa10837


## run it
with a simple `with` context. the `__enter__` method calls the `run` command of the client so we can just call `pass` so that nothing else happens after we exit the `run` method before the we `__exit__` the clients and the connections are closed.

In [11]:
with text_client:
    pass

select a command:
                	0	E(x)it
                	1	publish note
                	2	get last 10 notes by you
                	3	get last 10 from hex of author
                	4	delete an event
                	5	check deletions
                	6	get metadata by hex of user
                	7	check event
                
exiting


In [16]:
test = text_client.message_store

In [17]:
test['wss://nostr-2.zebedee.cloud:260012d4b5ca660bca4f945d5c2cba0292c1a3f4a503244ba7a98a745cadd3cb'].event.to_json_object()

{'id': '260012d4b5ca660bca4f945d5c2cba0292c1a3f4a503244ba7a98a745cadd3cb',
 'pubkey': '2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3',
 'created_at': 1672103350,
 'kind': 1,
 'tags': [],
 'content': 'testing again',
 'sig': '5be18991da6fc7d043a1793e4da1e5ac937964a496db82cfbc2c230ec6c3160b43ead76d45d15d1154ac8402c6be98503971c5d13678964a4611d251f4bfb089'}

In [9]:
test['wss://nostr-2.zebedee.cloud:260012d4b5ca660bca4f945d5c2cba0292c1a3f4a503244ba7a98a745cadd3cb'].event.to_json_object()

{'id': '260012d4b5ca660bca4f945d5c2cba0292c1a3f4a503244ba7a98a745cadd3cb',
 'pubkey': '2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3',
 'created_at': 1672103350,
 'kind': 1,
 'tags': [],
 'content': 'testing again',
 'sig': '5be18991da6fc7d043a1793e4da1e5ac937964a496db82cfbc2c230ec6c3160b43ead76d45d15d1154ac8402c6be98503971c5d13678964a4611d251f4bfb089'}

In [ ]:
raise

In [14]:

test['wss://nostr-2.zebedee.cloud:422c574a74f7668a38704f5b41dec69e3d18c91c3ebe14e1851ae9d54ee8ae02'].event.to_json_object()

{'id': '422c574a74f7668a38704f5b41dec69e3d18c91c3ebe14e1851ae9d54ee8ae02',
 'pubkey': '2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3',
 'created_at': 1672103350,
 'kind': 5,
 'tags': [['e',
   '260012d4b5ca660bca4f945d5c2cba0292c1a3f4a503244ba7a98a745cadd3cb']],
 'content': 'needs to be deleted',
 'sig': 'd3bd42f9459a4ee18f2d7eab1315acbd09077962846bf6faa56d7b929666fa54050d0c68ef7384bd763e6870c7b62713616e90f0732d7432636b432c8809ba7a'}

In [10]:
assert '260012d4b5ca660bca4f945d5c2cba0292c1a3f4a503244ba7a98a745cadd3cb' == '260012d4b5ca660bca4f945d5c2cba0292c1a3f4a503244ba7a98a745cadd3cb'